In [23]:
import pandas as pd
import numpy as np
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)
from datetime import timedelta


def interpolation(X1,X2,Y1,Y2,XInconnu):
    #prevent division by 0
    if(X2 == X1):
        return Y1
    YInconnu = Y2*((XInconnu-X1)/(X2-X1))+Y1*((X2-XInconnu)/(X2-X1))
    return YInconnu

# def interpolation2(X1,X2,Y1,Y2,XInconnu):
#     X = pd.Index(df['total_seconds'])
#     Y = pd.Index(df['tmoy'])

#      

#     print("Value of X")
#     print(X)
#     print(np.asarray(X).shape)

#      

#     print("\n value of Y")
#     print(Y)
#     print(np.asarray(Y).shape)

#     # test value => total_seconds
#     interpolate_x = 43200
#     print(interpolate_x)

#     # Finding the interpolation
#     y_interp = interp1d(X, Y)
#     print("Value of Y at x = {} is".format(interpolate_x), y_interp(interpolate_x))

#      

#     print( )
#     print(interpolate_x)
#     print(y_interp(interpolate_x))



def interpol_line(line1:pd.core.series.Series, line2:pd.core.series.Series, col2:str, target:int, toInterpol:list[str]=[], toNotInterpol:list[str]=[])->pd.core.series.Series:
    #col2 : colonne de temps 
    #target : temps de la nouvelle ligne
    #les temps doivent être des int ou float, peu importe l'échelle

    x1 = line1[col2]
    x2 = line2[col2]
    res = line1.copy()
    for col in res.index:
        res[col] = np.nan 
    res[col2] = target
    # interpolate all ints in the lines except col2
    for col in line1.index:
        if(col != col2  ):
            #check that col is numeric and not boolean
            if(pd.api.types.is_numeric_dtype(line1[col]) and  not pd.api.types.is_bool_dtype(line1[col])):
                if(col in toInterpol or col not in toNotInterpol):
                    y1 = line1[col]
                    y2 = line2[col]
                    print("interpolating", col, "from", y1, "to", y2, "for", target, "between", x1, "and", x2) 
                    print()
                    res[col]=(interpolation(x1, x2, y1, y2, target))
                    print("result", res[col])
    return res

def normalize(val, min, max):
    return (val-min)/(max-min)
    #FIX : floating point error

def denormalize(val, min, max):
    return val*(max-min)+min

def interpolDf(df, timecol, pas_voulu, offset=0, toInterpol:list[str]=[], toNotInterpol:list[str]=[]):
    #la fonction recale un dataset sur une échelle de temps régulière

    #df : dataframe à interpoler
    #timecol : colonne de temps
    #toInterpol : liste des colonnes à interpoler (facultatif)
    #toNotInterpol : liste des colonnes à ne pas interpoler (facultatif)
    #pas_voulu : pas voulu pour l'interpolation
    #offset : décalage à appliquer à la colonne de temps
    #les temps doivent être des int ou float, peu importe l'échelle
  
    #on trie le df par temps
    df = df.sort_values(by=timecol)

    #on récupère les temps min et max
    min = df[timecol].min()
    max = df[timecol].max() 

    #on normalise les temps et on arrondit a 5 chiffres après la virgule
    df[timecol] = df[timecol].apply(lambda x: normalize(x, min, max))
    df[timecol] = df[timecol].apply(lambda x: round(x, 5))

    #on crée une colonne original pour marquer les lignes originales et une colonne filled pour marquer les lignes interpolables
    df['original'] = True
    df['filled'] = True


    #on adapte le pas voulu et l'offset à la normalisation
    pas_voulu = pas_voulu/(max-min)
    offset = offset/(max-min)

    #on crée une liste de temps à interpoler arrondis à 5 chiffres après la virgule
    temps = np.arange(min + offset, 1, pas_voulu)
    temps = [round(t, 5) for t in temps]

    #on ajoute des lignes vides avec les temps à interpoler au df si elles n'existent pas déjà
    for t in temps:  
        if(t not in df[timecol].values): #warning penser a floating point error
            newline = pd.Series([np.nan for i in range(len(df.columns))], index=df.columns)
            newline[timecol] = t
            newline['original'] = False 
            newline['filled'] = False
            df = df.append(newline, ignore_index=True)
        
    
    #on trie le df par temps
    df = df.sort_values(by=timecol)
    df = df.reset_index(drop=True)
    print(df)
    #on interpole les lignes vides
    for i in range(len(df)):
        if(df['filled'][i] == False):
            #on récupère les lignes avant et après si elles existent
    
            line1 = None
            line2 = None

            #debug prints
            

            print()
            print('finding line 2')
            for j in range(i+1, len(df)):
                # print('j',j)
                # print('line j',df.iloc[j])
                if(df['filled'][j] == True):
                    line2 = df.iloc[j]
                    print('line 2 found')
                    break
                # else:
                #     print('line 2 NOT found')
                    
                    
            for k in range(i-1, -1, -1):
                # print('line at k',df.iloc[k])
                if(df['filled'][k] == True):
                    line1 = df.iloc[k]
                    print('line 1 found')
                    break
                # else:
                #     print('line 1 NOT found')
                    
                    

            #on interpole la ligne vide
            # print('LINE1',line1)
            # print('LINE2',line2)
            if(line1 is None or line2 is None):
                print("error : no line before or after line at", i)
                continue
            df.iloc[i] = interpol_line(line1, line2, timecol, df[timecol][i], toInterpol, toNotInterpol)
            df['filled'][i] = True


    #on dénormalise les temps
    df[timecol] = df[timecol].apply(lambda x: denormalize(x, min, max))

    #on supprime les lignes originales ?
    # df = df[~df['original']]

    #on supprime les colonne original et filled
    df = df.drop(columns=['original'])
    df = df.drop(columns=['filled'])


    return df



In [24]:


def df_generate_int(t_start , t_end, period, T_fld_name ='date_time') :
    #pour un entier en temps
    times = []
    for t in range(t_start, t_end, period):
        times.append(t)
    df = pd.DataFrame(times, columns=[T_fld_name])
    return df




def df_generate_datetime(t_start , t_end, period, T_fld_name ='date_time') :
    #pour un temps en format datetime

    #on convertit tout en secondes
    start = int(t_start.timestamp())
    end = int(t_end.timestamp())
    step = int(period.total_seconds())

    times = []
    for t in range (start, end, step):
        times.append(t)

    # create an empty list to store the UTC datetime objects
    utc_times = []

    # convert the timestamps to datetime objects in UTC timezone
    for t in times:
        utc_times.append(pd.to_datetime(t, unit='s', utc=True))

    # create an empty list to store the Europe/Paris datetime objects
    paris_times = []

    # convert the UTC datetime objects to Europe/Paris timezone and remove the timezone information
    for t in utc_times:
        paris_time = t.astimezone(tz='Europe/Paris')
        paris_time = paris_time.replace(tzinfo=None)
        paris_times.append(paris_time)

    df = pd.DataFrame(paris_times, columns=[T_fld_name])
    return df

#test 
# df = df_generate(0, 100, 1)
# print(df)
# df = df_generate_datetime(datetime.datetime(2021, 1, 1), datetime.datetime(2021, 1, 20, tzinfo=datetime.timezone.utc), timedelta(days=1))
# print(df)


In [79]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


def df_interpolation(dataframe1, T_fld_name1, dataframe2, T_fld_name2, fld_name) :
    # on prend 2 df complets, on ajoute la colonne fld_name a dataframe1 et on l'interpole en gardant les valeurs de dataframe2

    # dataframe1 : données réelles
    # T_fld_name1 : nom de la colonne de temps de dataframe1
    # dataframe2 : df avec les temps et les données de  fld_name à interpoler
    # T_fld_name2 : nom de la colonne de temps de dataframe2
    # fld_name : nom de la colonne à insérer dans dataframe1


    df = dataframe1.copy()

    df2 = dataframe2[[T_fld_name2, fld_name]].copy()
    
    print('DATAFRAMES')
    print(df)
    print(df2)

    if(fld_name in df.columns):
        return df.merge(df2, how='left', left_on=T_fld_name1, right_on=T_fld_name2)
    
    df[fld_name] = np.NaN
   
    if(type(df[T_fld_name1][0]) != type(df2[T_fld_name2][0])):
        print('error : different types of time')
        return
    
    

    #add all values of df2[T_fld_name2] to df[T_fld_name1] if they don't exist
    # for row,index in df2.iterrows():
    #     if(df2[T_fld_name2][row] not in df[T_fld_name1].values):
    #         df.loc[len(df)] = [df2[T_fld_name2][row], np.NaN]
    #         print('new value added to df : ', df2[T_fld_name2][row])
             #BROKEN

    # print('MERGED DATES')
    # print(df)

    df = df.sort_values(by=[T_fld_name1])
    df = df.reset_index(drop=True)

    if(T_fld_name1 != T_fld_name2):
        df = df.drop(columns=[T_fld_name2])
        
    #on insere les valeurs de df2 dans df si le temps correspond
    for row,index in df.iterrows():
        bigdf2date = df2.loc[df2[T_fld_name2] == df[T_fld_name1][row]]
        # print('df2date',bigdf2date)
        # print()
        # print(type(bigdf2date))
        if(len(bigdf2date) == 0):
            continue
        df2date = bigdf2date.iloc[0]
        # print('CHANGE')
        # print('df2date',df2date)
        # print()
        # print(type(df2date))
        df[fld_name][row] = df2date[fld_name]
        # print('found matching date : ', df[T_fld_name1][row], 'value : ', df2date[fld_name])


    # print_full(df)

    
    #on interpole les valeurs manquantes

    for i in range(len(df)):
        if(np.isnan(df[fld_name][i])):
            #on récupère les lignes avant et après si elles existent
    
            line1 = None
            line2 = None

          
            # print()
            # print('finding lines')

            for j in range(i+1, len(df)):
                # print('j',j)
                # print('j date and value',df.iloc[j][T_fld_name1], df.iloc[j][fld_name])
                if(not np.isnan(df[fld_name][j])):
                    line2 = df.iloc[j]
                    # print('line 2 found')
                    break
                # else:
                    # print('line 2 NOT found')
                    
                    
            for k in range(i-1, -1, -1):
                # print('line at k',df.iloc[k])
                if(not np.isnan(df[fld_name][k])):
                    line1 = df.iloc[k]
                    # print('line 1 found')
                    break
                # else:
                #     print('line 1 NOT -((ggg))')
                    
                    

            #on interpole la ligne vide
            # print('LINE1',line1)
            # print('LINE2',line2)
            if(line1 is None or line2 is None):
                print("error : no line before or after line at", i)
                continue
            # print(line1[fld_name])
            # print(line2[fld_name])
            # print(line1[T_fld_name1])
            # print(line2[T_fld_name1])
            # print(df[T_fld_name1][i])

            #convert date strings to ints
            if(type(line1[T_fld_name1]) == str):
                timeline1 = datetime.datetime.strptime(line1[T_fld_name1], '%Y-%m-%d')
            if(type(line2[T_fld_name1]) == str):
               timeline2 = datetime.datetime.strptime(line2[T_fld_name1], '%Y-%m-%d')
            if(type(df[T_fld_name1][i]) == str):
                timelinedf = datetime.datetime.strptime(df[T_fld_name1][i], '%Y-%m-%d')

            # print(line1[fld_name])
            # print(line2[fld_name])
            # print(timeline1)
            # print(timeline2)
            # print(timelinedf)

            value = interpolation(timeline1, timeline2, line1[fld_name], line2[fld_name], timelinedf)
                        # value = interpolation( line1[T_fld_name1], line2[T_fld_name1], line1[fld_name], line2[fld_name], df[T_fld_name1][i])

            df[fld_name][i] = value
            # print('line',i,'interpolated value :',value)
        else:
            pass
            # print('line',i,'is not empty')
            # print('value :',df[fld_name][i])
    return df    

# A TESTER



In [80]:
#test
df2 =  pd.read_csv('df1.csv')
df1 =  pd.read_csv('df2.csv')
df1.head()
df2.head()

dfres = df_interpolation(df1,'date',df2,'date','temp')
print_full(dfres)

DATAFRAMES
           date  wind  powergen
0    2020-01-01     5      1666
1    2020-01-02    10      3333
2    2020-01-03    13      4300
3    2020-01-04    28      9333
4    2020-01-05    25      8333
..          ...   ...       ...
117  2020-04-27    18      6000
118  2020-04-28     2       666
119  2020-04-29     5      1666
120  2020-04-30    22      7333
121  2020-05-01     9      3000

[122 rows x 3 columns]
         date  temp
0  2020-01-01     1
1  2020-02-01    -4
2  2020-03-01    12
3  2020-04-01    18
4  2020-05-01    20
5  2021-01-01     2


C:\Users\AlassaneSY\AppData\Local\Temp\ipykernel_16580\65157343.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[fld_name][row] = df2date[fld_name]
C:\Users\AlassaneSY\AppData\Local\Temp\ipykernel_16580\65157343.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[fld_name][i] = value


           date  wind  powergen       temp
0    2020-01-01     5      1666   1.000000
1    2020-01-02    10      3333   0.838710
2    2020-01-03    13      4300   0.677419
3    2020-01-04    28      9333   0.516129
4    2020-01-05    25      8333   0.354839
5    2020-01-06     4      1333   0.193548
6    2020-01-07    27      9000   0.032258
7    2020-01-08    20      6666  -0.129032
8    2020-01-09    29      9666  -0.290323
9    2020-01-10    11      3666  -0.451613
10   2020-01-11    25      8333  -0.612903
11   2020-01-12     5      1666  -0.774194
12   2020-01-13    17      5666  -0.935484
13   2020-01-14    27      9000  -1.096774
14   2020-01-15    22      7333  -1.258065
15   2020-01-16    28      9333  -1.419355
16   2020-01-17     2       666  -1.580645
17   2020-01-18     0         0  -1.741935
18   2020-01-19    23      7666  -1.903226
19   2020-01-20     1        33  -2.064516
20   2020-01-21    17      5666  -2.225806
21   2020-01-22    22      7333  -2.387097
22   2020-0